# 텐서와 그래프 실행

In [1]:
# 텐서플로 import
import tensorflow as tf

In [2]:
# 상수를 hello 변수에 저장
hello = tf.constant('Hello, TensorFlow!')
print(hello)

Tensor("Const:0", shape=(), dtype=string)


hello가 텐서플로의 **텐서** 라는 자료형을 담고 있다
**텐서는** 텐서플로에서 다양한 수학식을 계산하기 위한 가장 기본적이고 중요한 자료형이다.
다음과 같이 **랭크(Rank)** 와 **셰이프(Shape)** 라는 개념을 가지고 있다.

</bn>

```python
3                              # 랭크가 0인 텐서, 셰이프는 []
[1., 2., 3.]                   # 랭크가 1인 텐서, 셰이프는 [3]
[[1., 2., 3.], [4., 5., 6.]]   # 랭크가 2인 텐서, 셰이프는 [2, 3]
```
> 텐서 자료형의 형태는 배열과 비슷하다고 생각하면 된다.

</bn>

**dtype** 은 해당 텐서에 담긴 요소들의 자료형이다. ex) string, float, int


In [3]:
# 텐서를 이용한 덧셈
a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a, b)
print(c)

Tensor("Add:0", shape=(), dtype=int32)


42가 나올 것으로 생각할 수 있지만, 위와 같이 텐서의 형태로 출력된다.
왜냐하면 텐서플로 프로그램의 구조는 **그래프의 생성과 그래프 실행** 으로 구성되어 있다.

<br>

* **그래프** : 텐서들의 연산 모음. 연산이 필요할 때만 실행하는 코드를 넣어 <u>원하는 시점</u> 에 실제 연산을 수행하도록 한다.
* **그래프 실행** : Session 객체와 run 메소드를 이용하면 된다.

In [4]:
sess = tf.Session()

print(sess.run(hello))
print(sess.run([a, b, c]))

sess.close

b'Hello, TensorFlow!'
[10, 32, 42]


<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x1351565c0>>

In [5]:
print(type(sess.run(hello)))
print(type(sess.run([a, b, c])))

<class 'bytes'>
<class 'list'>


# 플레이스홀더와 변수
**플레이스홀더** : 그래프에 사용할 입력값을 나중에 받기 위해 사용하는 매개변수

**변수** : 그래프를 최적화하는 용도로 텐서플로가 학습한 결과를 갱신하기 위해 사용하는 변수

In [6]:
# 플레이스홀더 사용
X = tf.placeholder(tf.float32, [None, 3])
print(X)

Tensor("Placeholder:0", shape=(?, 3), dtype=float32)


In [7]:
# X에 넣을 자료 정의
x_data = [[1, 2, 3], [4, 5, 6]]

In [13]:
# 변수 정의(W와 b의 값은 바뀔 수 있다)
# tf.random_normal : 정규분포의 무작위 값으로 초기화
W = tf.Variable(tf.random_normal([3, 2]))
b = tf.Variable(tf.random_normal([2, 1]))

In [18]:
print(W)
print(b)

<bound method RefVariable.value of <tf.Variable 'Variable_2:0' shape=(3, 2) dtype=float32_ref>>
<tf.Variable 'Variable_3:0' shape=(2, 1) dtype=float32_ref>


In [19]:
# 행렬 곱셈 실행
expr = tf.matmul(X, W) + b

텐서플로로 딥러닝을 하기 위해서는 **행렬곱** 정의를 알아야 한다.
* 행렬곱 A x B 에 대하여, 행렬 A의 열 수와 행렬 B의 행 수는 같아야 한다.
* 행렬곱 A x B 를 계산한 행렬 AB의 크기는 A의 행 개수와 B의 열 개수가 된다.

<br>

<img src="https://t1.daumcdn.net/cfile/tistory/2433A83D566050DA1A">

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 정의한 변수들을 초기화

print("=== x_data ===")
print(x_data)
print("=== W ===")
print(sess.run(W))
print("=== b ===")
print(sess.run(b))
print("=== expr ===")
# feed_dict : 그래프를 실행할 때 사용할 입력값을 지정
print(sess.run(expr, feed_dict={X: x_data}))

sess.close

=== x_data ===
[[1, 2, 3], [4, 5, 6]]
=== W ===
[[ 1.0148433  -1.4060212 ]
 [ 0.25959322 -1.3226057 ]
 [ 0.83300525  0.68061125]]
=== b ===
[[ 0.30882627]
 [-2.0100303 ]]
=== expr ===
[[  4.341872   -1.7005727]
 [  8.345341  -10.163477 ]]


<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x1393ef390>>

# 선형 회귀 모델 구현하기
**선형 회귀** : 주어진 x 와 y 값을 가지고 서로 간의 관계를 파악하는 것입니다. x 값이 주어졌을 때 y 값을 쉽게 알 수 있다.

<br>
    
x_data 와 y_data의 상관관계를 파악해보자.

In [21]:
x_data = [1, 2, 3]
y_data = [1, 2, 3]

In [22]:
# -1.0 ~ 1.0 사이의 균등분포를 가진 무작위 값으로 초기화
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

In [23]:
# 자료를 입력받을 플레이스홀더를 설정
# name 매개변수로 플레이스홀더의 이름을 설정한다.
X = tf.placeholder(tf.float32, name="X")
Y = tf.placeholder(tf.float32, name="Y")

print(X)
print(Y)

Tensor("X:0", dtype=float32)
Tensor("Y:0", dtype=float32)


In [24]:
# X와 Y의 상관관계(선형관계)를 분석하기 위한 수식 작성
hypothesis = W * X + b

이 수식은 W와의 곱과 b와의 합을 통해 **X와 Y의 관계를** 설명하겠다는 뜻이다.
즉, X가 주어졌을 때 Y를 만들어 낼 수 있는 W와 b를 찾아내겠다는 의미이다.
* **W** : 가중치
* **b** : 편향
> 이 수식은 선형 회귀는 물론 신경망 학습에 가장 기본이 되는 수식이다.

<br>

손실 함수를 작성해보자.
* **손실 함수(loss function)** : 한 쌍(x,y)의 데이터에 대한 **손실값**을 계산하는 함수이다.
* 손실값 : 실제값과 모델로 예측한 값이 얼마나 차이가 나는지를 나타내는 값
> 손실값이 작을수록 X와 Y의 관계가 잘 설명되고 있는 것이며, X 값에 대한 Y 값을 정확하게 예측할 수 있다는 뜻이다.
* **비용(cost)** : 손실을 전체 데이터에 대해 구한 경우
> 즉 **학습**이란 변수들의 값을 다양하게 넣어 계산해보면서 이 손실값을 최소화하는 것이다.

In [25]:
# 손실값 = (예측값 - 실제값)^2
# 모든 데이터에 대한 손실값의 평균을 낸다.
cost = tf.reduce_mean(tf.square(hypothesis - Y))

텐서플로 함수을 볼 수 있는 사이트
[https://www.tensorflow.org/api_docs/](https://www.tensorflow.org/api_docs/)

<br>

## 경사하강법 (gradient descent)
최적화 함수를 이용해 손실값을 최소화하는 연산 그래프 생성
* **최적화 함수** : 가중치와 편향 값을 변경해가면서 손실값을 최소화하는 가장 최적화된 가중치와 편향 값을 찾아주는 함수.
* **경사하강법** : 빠르게 최적화하기 위한, 즉 빠르게 학습하기 위한 다양한 방법 중에 가장 기본적인 알고리즘이다.
* **learning_rate(학습률)** : 학습을 얼마나 '급하게' 할 것인가를 설정하는 값
  * 값이 너무 작으면 학습 속도가 매우 느려지고, 너무 크면 최적의 손실값을 찾지 못한다.
* **하이퍼파라미터(hyperparameter)** : 학습을 진행하는 과정에 영향을 주는 변수. 이 값에 따라 학습 속도나 신경망 성능이 크게 달라진다.

In [36]:
# 최적화 함수를 이용해 손실값을 최소화하는 연산 그래프 생성 
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train_op = optimizer.minimize(cost)

In [41]:
# 세션을 생성하고 변수들 초기화
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 최적화 수행 그래프 실행, 실행 시마다 변화하는 손실값 출력
    for step in range(1000):
        # feed_dict 를 통해 x_data 와 y_data 의 상관관계를 알아낸다
        _, cost_val = sess.run([train_op, cost], feed_dict={X: x_data, Y: y_data})
        
        print(step, cost_val, sess.run(W), sess.run(b))
        
    # X 값을 넣고 결과 확인
    print("\nX: 5, Y: ", sess.run(hypothesis, feed_dict={X: 5}))
    print("X: 2.5, Y: ", sess.run(hypothesis, feed_dict={X: 2.5}))
        
# 결과를 보면 손실값이 점점 줄어드는것을 확인할 수 있다.
# 그리고, X가 5일 때는 Y 값으로 5를 2.5일 때는 2.5를 정확히 예측해내는것을 확인할 수 있다

0 0.47906062 [1.216896] [-0.4156183]
1 0.031692866 [1.1807071] [-0.41925305]
2 0.025115376 [1.1797483] [-0.40768528]
3 0.023861783 [1.1750573] [-0.39804754]
4 0.022727607 [1.1708895] [-0.38846093]
5 0.021648029 [1.166777] [-0.37912455]
6 0.0206197 [1.1627684] [-0.37001044]
7 0.019640274 [1.1588554] [-0.3611157]
8 0.018707346 [1.1550367] [-0.35243472]
9 0.017818745 [1.1513096] [-0.34396246]
10 0.016972335 [1.1476723] [-0.33569384]
11 0.016166141 [1.1441224] [-0.327624]
12 0.015398219 [1.1406578] [-0.31974813]
13 0.014666797 [1.1372765] [-0.3120616]
14 0.013970117 [1.1339765] [-0.3045599]
15 0.013306528 [1.1307558] [-0.2972385]
16 0.012674446 [1.1276125] [-0.2900931]
17 0.012072399 [1.1245447] [-0.28311944]
18 0.011498973 [1.1215507] [-0.27631345]
19 0.010952753 [1.1186287] [-0.26967105]
20 0.0104325 [1.115777] [-0.26318833]
21 0.009936943 [1.1129938] [-0.25686148]
22 0.0094649345 [1.1102775] [-0.25068673]
23 0.009015341 [1.1076266] [-0.24466039]
24 0.008587111 [1.1050392] [-0.23877895]


224 5.0911177e-07 [1.0008087] [-0.00183852]
225 4.8491984e-07 [1.0007893] [-0.00179431]
226 4.619153e-07 [1.0007703] [-0.00175116]
227 4.3992614e-07 [1.0007519] [-0.00170906]
228 4.1900543e-07 [1.0007337] [-0.00166798]
229 3.990715e-07 [1.0007161] [-0.00162787]
230 3.8009398e-07 [1.0006989] [-0.00158872]
231 3.6200777e-07 [1.0006821] [-0.00155053]
232 3.4494383e-07 [1.0006657] [-0.00151328]
233 3.2853245e-07 [1.0006497] [-0.00147689]
234 3.1286825e-07 [1.0006341] [-0.00144138]
235 2.9796226e-07 [1.0006188] [-0.00140672]
236 2.8385804e-07 [1.0006039] [-0.0013729]
237 2.7043612e-07 [1.0005894] [-0.00133989]
238 2.5754798e-07 [1.0005753] [-0.00130766]
239 2.4529425e-07 [1.0005615] [-0.00127625]
240 2.3367187e-07 [1.0005479] [-0.00124559]
241 2.2257706e-07 [1.0005348] [-0.00121563]
242 2.11976e-07 [1.0005219] [-0.00118641]
243 2.0193905e-07 [1.0005094] [-0.00115789]
244 1.9233313e-07 [1.0004971] [-0.00113007]
245 1.8318401e-07 [1.0004852] [-0.00110289]
246 1.7448205e-07 [1.0004735] [-0.001

439 1.4875212e-11 [1.0000043] [-9.837839e-06]
440 1.4017824e-11 [1.0000042] [-9.587499e-06]
441 1.30088535e-11 [1.0000042] [-9.33716e-06]
442 1.2468841e-11 [1.000004] [-9.134505e-06]
443 1.2141991e-11 [1.0000039] [-8.931849e-06]
444 1.1941855e-11 [1.0000038] [-8.733167e-06]
445 1.0834593e-11 [1.0000038] [-8.5066695e-06]
446 1.0351424e-11 [1.0000037] [-8.3278555e-06]
447 9.800753e-12 [1.0000036] [-8.129174e-06]
448 9.4360075e-12 [1.0000035] [-7.938439e-06]
449 9.3377155e-12 [1.0000033] [-7.743731e-06]
450 8.763361e-12 [1.0000033] [-7.5371013e-06]
451 7.929657e-12 [1.0000032] [-7.362261e-06]
452 7.640703e-12 [1.0000032] [-7.1635786e-06]
453 7.2238513e-12 [1.0000031] [-7.01258e-06]
454 7.285432e-12 [1.000003] [-6.8536347e-06]
455 6.770288e-12 [1.000003] [-6.682768e-06]
456 6.617521e-12 [1.0000029] [-6.547664e-06]
457 6.3190933e-12 [1.0000027] [-6.3887182e-06]
458 5.6381566e-12 [1.0000027] [-6.1940095e-06]
459 5.366966e-12 [1.0000027] [-6.043011e-06]
460 5.2355156e-12 [1.0000026] [-5.92777

667 3.7895614e-14 [1.0000004] [-6.229629e-07]
668 3.7895614e-14 [1.0000004] [-6.229629e-07]
669 3.7895614e-14 [1.0000004] [-6.229629e-07]
670 3.7895614e-14 [1.0000004] [-6.229629e-07]
671 3.7895614e-14 [1.0000004] [-6.229629e-07]
672 3.7895614e-14 [1.0000004] [-6.229629e-07]
673 3.7895614e-14 [1.0000004] [-6.229629e-07]
674 3.7895614e-14 [1.0000004] [-6.229629e-07]
675 3.7895614e-14 [1.0000004] [-6.229629e-07]
676 3.7895614e-14 [1.0000004] [-6.229629e-07]
677 3.7895614e-14 [1.0000004] [-6.229629e-07]
678 3.7895614e-14 [1.0000004] [-6.229629e-07]
679 3.7895614e-14 [1.0000004] [-6.229629e-07]
680 3.7895614e-14 [1.0000004] [-6.229629e-07]
681 3.7895614e-14 [1.0000004] [-6.229629e-07]
682 3.7895614e-14 [1.0000004] [-6.229629e-07]
683 3.7895614e-14 [1.0000004] [-6.229629e-07]
684 3.7895614e-14 [1.0000004] [-6.229629e-07]
685 3.7895614e-14 [1.0000004] [-6.229629e-07]
686 3.7895614e-14 [1.0000004] [-6.229629e-07]
687 3.7895614e-14 [1.0000004] [-6.229629e-07]
688 3.7895614e-14 [1.0000004] [-6.

894 3.7895614e-14 [1.0000004] [-6.229629e-07]
895 3.7895614e-14 [1.0000004] [-6.229629e-07]
896 3.7895614e-14 [1.0000004] [-6.229629e-07]
897 3.7895614e-14 [1.0000004] [-6.229629e-07]
898 3.7895614e-14 [1.0000004] [-6.229629e-07]
899 3.7895614e-14 [1.0000004] [-6.229629e-07]
900 3.7895614e-14 [1.0000004] [-6.229629e-07]
901 3.7895614e-14 [1.0000004] [-6.229629e-07]
902 3.7895614e-14 [1.0000004] [-6.229629e-07]
903 3.7895614e-14 [1.0000004] [-6.229629e-07]
904 3.7895614e-14 [1.0000004] [-6.229629e-07]
905 3.7895614e-14 [1.0000004] [-6.229629e-07]
906 3.7895614e-14 [1.0000004] [-6.229629e-07]
907 3.7895614e-14 [1.0000004] [-6.229629e-07]
908 3.7895614e-14 [1.0000004] [-6.229629e-07]
909 3.7895614e-14 [1.0000004] [-6.229629e-07]
910 3.7895614e-14 [1.0000004] [-6.229629e-07]
911 3.7895614e-14 [1.0000004] [-6.229629e-07]
912 3.7895614e-14 [1.0000004] [-6.229629e-07]
913 3.7895614e-14 [1.0000004] [-6.229629e-07]
914 3.7895614e-14 [1.0000004] [-6.229629e-07]
915 3.7895614e-14 [1.0000004] [-6.